#**Food Recovery Cost per Pickup & Pound**
This Notebook contains skeleton code for calculating Food Recovery Costs for the operations designated in CA Senate Bill 1383.


####Data Needed:
 - The distance covered (miles, round trip from organization facility)
 - Type of vehicle (to calculate fuel efficiency (1/MPG))
 - Estimated gas prices ($)
 - The amount of food transported (lbs)

#**Setup and Data Cleaning**

##**Import Basic Packages & Files**

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import os
drive.mount('/content/drive')
shared_drive = '/content/drive/My Drive/ACCFB/ACCFB Data'
print(os.listdir(shared_drive))

##**Select Primary Sheet**
From a personal or shared Google Drive

In [ ]:
def find_paths(root, name_substring):
    hits = []
    for r, d, f in os.walk(root):
        for nm in d + f:
            if name_substring.lower() in nm.lower():
                hits.append(os.path.join(r, nm))
    return hits

# Search across My Drive and Shared drives
hits = []
# hits += find_paths('/content/drive/My Drive', 'ACCFB')
hits += find_paths('/content/drive/Shared drives', 'ACCFB')
for p in hits:
    print(p)

In [ ]:
# Imports excel sheet and drops columns and rows that are not part of data
excel_path = '/content/drive/My Drive/ACCFB/ACCFB Data/(with addresses) Pickup Data for UCB _11.6-2025-11-06-17-01-48.xlsx'
df = pd.read_excel(excel_path, sheet_name='Pickup Data for UCB _11.6', header = 12)
print(df.shape)
print(df.head())

####**Fill Empty Rows with Correct Values**

In [ ]:
# Forward-fill specific columns by name
cols_to_ffill = [
    'Agency: Account Name  ↑',
    'Food Donor  ↑',
    'FRP Donor Address  ↑',
    'Agency Address'
]
df[cols_to_ffill] = df[cols_to_ffill].ffill()
display(df.head())
print(df.shape)

##**Install GoogleMaps and Insert API Key**
Generate a Google Maps API and enable the [Distance Matrix API](console.cloud.google.com/google/maps-apis/api-list).

In [ ]:
!pip install googlemaps

In [ ]:
import googlemaps
from google.colab import userdata
API_KEY = userdata.get('PlacesAPI')
gmaps = googlemaps.Client(key=API_KEY)

##**Select Neccessary Columns & Filter for Unique Routes**

In [ ]:
df.columns = df.columns.str.replace('\s+', ' ', regex=True).str.strip()

# After cleaning, column names should now have a single space before '↑'.
# Adjust selection and mapping accordingly.
columns_to_select = ['Agency: Account Name ↑', 'Agency Address', 'Food Donor ↑', 'FRP Donor Address ↑', 'Total Quantity']
driving_dat = df[columns_to_select].drop_duplicates(subset=['Agency: Account Name ↑', 'FRP Donor Address ↑']).iloc[:-3]

driving_data_columns = {
    'Agency: Account Name ↑': 'Agency',
    'Food Donor ↑': 'Food Donor',
    'FRP Donor Address ↑': 'Donor Address',
    'Total Quantity': 'Total Quantity (lbs)'
}

driving_data = driving_dat.rename(columns=driving_data_columns)
display(driving_data)

#**Calculate Driving Distance and Duration**
1. Define a function to get Driving Distance and Duration from two addresses
2. Apply the function to the Agency and Food Donor Address colummns

In [ ]:
#Function to get Driving Distance and Duration between two addresses using the Distance Matrix API from Google Maps
## Takes 2 arguments: starting address, ending address
### Returns Driving Distance (meters) and Driving Duration (seconds)

def get_distance_duration(origin_address, destination_address):
    try:
        response = gmaps.distance_matrix(origin_address, destination_address, mode='driving')
        if response['status'] == 'OK' and response['rows'][0]['elements'][0]['status'] == 'OK':
            distance_meters = response['rows'][0]['elements'][0]['distance']['value']
            duration_seconds = response['rows'][0]['elements'][0]['duration']['value']
            return distance_meters, duration_seconds
        else:
            print(f"API response not OK or no route found for {origin_address} to {destination_address}. Status: {response['rows'][0]['elements'][0]['status'] if response['rows'][0]['elements'][0] else 'No Elements'}")
            return None, None
    except Exception as e:
        print(f"Error calling Google Maps API for {origin_address} to {destination_address}: {e}")
        return None, None

print("Defined get_distance_duration function.")

In [ ]:
distances = []
durations = []

for index, row in driving_data.iterrows():
    origin = row['Agency Address']
    destination = row['Donor Address']
    distance_meters, duration_seconds = get_distance_duration(origin, destination)
    distances.append(distance_meters)
    durations.append(duration_seconds)

driving_data['Driving Distance (meters)'] = distances
driving_data['Driving Duration (seconds)'] = durations

#  Multiply by 2 for round trip, Convert distance from meters to miles (1 meter = 0.000621371 miles)
driving_data['Driving Distance (miles)'] = (driving_data['Driving Distance (meters)'] *2) * 0.000621371
# Multiply by 2 for round trip, Convert duration from seconds to minutes
driving_data['Driving Duration (minutes)'] = (driving_data['Driving Duration (seconds)'] * 2) / 60

driving_data.drop(columns=['Driving Distance (meters)', 'Driving Duration (seconds)'], inplace=True)

In [ ]:
display(driving_data.head())
print(driving_data.shape)

##**Add all Pickups Back**
This cell will match all rows to their pickup distances and durations for each route.

In [ ]:
# Select relevant columns from df, using corrected column names (single space before ↑)
# and the Total Quantity column. Apply .iloc[:-3] to exclude the last 3 rows.
columns_to_select_for_costs = [
    'Agency: Account Name ↑',
    'Agency Address',
    'Food Donor ↑',
    'FRP Donor Address ↑',
    'Total Quantity'
]
costs = df[columns_to_select_for_costs].iloc[:-3]

# Use the same renaming dictionary as before to standardize column names in 'costs'
# for consistency with 'driving_data' and clearer merging.
costs = costs.rename(columns=driving_data_columns)

# Define the columns to merge on. These columns are common identifiers for a route
# and exist in both 'costs' and 'driving_data'. By merging on these, they won't get _x or _y suffixes.
merge_on_cols = ['Agency', 'Agency Address', 'Food Donor', 'Donor Address']

# Perform the merge.
# 'Total Quantity (lbs)' is present in both `costs` (left) and `driving_data` (right)
# and is NOT in `merge_on_cols`. Thus, it will result in `Total Quantity (lbs)_x` and `Total Quantity (lbs)_y`.
# We want to keep the one from `costs` (left), which is `Total Quantity (lbs)_x`, as it represents individual pickup quantity.
costs = pd.merge(costs, driving_data, on=merge_on_cols, how='left')

# Drop the 'Total Quantity (lbs)_y' column and rename 'Total Quantity (lbs)_x'
if 'Total Quantity (lbs)_y' in costs.columns:
    costs.drop(columns=['Total Quantity (lbs)_y'], inplace=True)
if 'Total Quantity (lbs)_x' in costs.columns:
    costs.rename(columns={'Total Quantity (lbs)_x': 'Total Quantity (lbs)'}, inplace=True)

# The 'Cost Per Pickup ($)' column is added later, so this check won't trigger at this stage.
# Keeping it for robustness if the notebook structure changes later.
if 'Cost Per Pickup' in costs.columns:
    costs.drop(columns=['Cost Per Pickup'], inplace=True)

display(costs.head())

##**Calculating Cost-Per-Pickup: Variables & Function**

###**Mileage Cost for Multiple Vehicle Types based off Weight**
Use this cell for estimating vehicle type by weightage.
#
**Do not run the next cell if using this method.**

In [ ]:
#Cost per Pickup Function
##Takes Driving Distance (miles) as an Argument
gas = 4.85 ##Insert your own local gas price
def costperpick(miles, pounds):
  if pounds < 500:
    sedan_cost = miles * (1/33) * gas
    return sedan_cost
  if pounds < 700:
    suv_cost = miles * (1/26) * gas
    return suv_cost
  if pounds < 1000:
    truck_cost = miles * (1/23) * gas
    return truck_cost
  else:
    large_truck_cost = miles * (1/20) * gas
    return large_truck_cost

###**Mileage Cost for One Vehicle Type**
Use the values in this cell to pick your most used vehicle type. Assign the correct value to the 'invmpg' value in the cell.

Common Car Types Inverse Mileage:


**Do not run this cell if you used the previous method.**

In [ ]:
#Variables
invmpg = 1/25.4 ##Insert your mileage value
gas = 4.85 ##Insert your own local gas price

#Cost per Pickup Function
##Takes Driving Distance (miles) as an Argument
def costperpick(miles):
  cost = miles * invmpg * gas
  return cost

##**Calculating Cost-Per-Pickup: Application**

In [ ]:
costs['Cost Per Pickup ($)'] = costs.apply(lambda row: costperpick(row['Driving Distance (miles)'], row['Total Quantity (lbs)']), axis=1).round(2)
display(costs.head())
per_pickup = costs

In [ ]:
visual_columns = per_pickup[['Agency', 'Total Quantity (lbs)', 'Driving Distance (miles)', 'Driving Duration (minutes)', 'Cost Per Pickup ($)' ]]
display(visual_columns.head())

#**Calculating Cost-Per-Pound**

##**Define Cost per Pound Function**

In [ ]:
#Cost per Pound Function
##takes pickup_cost and pounds as arguments from per_pound table
###returns cost per pound ($)
def per_pound_cost(pickup_cost, pounds):
  return pickup_cost / pounds

##**Calculate Cost per Pound**
Call per_pound_cost function on 'Cost per Pickup' and 'Total Quantity' columns


In [ ]:
costs['Cost Per Pound ($)'] = costs.apply(lambda row: per_pound_cost(row['Cost Per Pickup ($)'], row['Total Quantity (lbs)']), axis=1).round(6)
display(costs.head())

In [ ]:
vis_columns = costs[['Agency', 'Total Quantity (lbs)', 'Driving Distance (miles)', 'Driving Duration (minutes)', 'Cost Per Pickup ($)', 'Cost Per Pound ($)' ]]
display(vis_columns.head())


mean_cost_per_pound = costs['Cost Per Pound ($)'].mean()
median_cost_per_pound = costs['Cost Per Pound ($)'].median()
range_cost_per_pound = costs['Cost Per Pound ($)'].max() - costs['Cost Per Pound ($)'].min()

print(f"Mean Cost Per Pound ($): {mean_cost_per_pound:.4f}")
print(f"Median Cost Per Pound ($): {median_cost_per_pound:.4f}")
print(f"Range of Cost Per Pound ($): {range_cost_per_pound:.4f}")

##**Calculating Weighing Time**

In [ ]:
#Weighing Time Function
##takes total pounds per pickup, returns labor cost.
def weigh_time(pounds):
  weighing_time = (0.002 * pounds) + 0.25
  return weighing_time

In [ ]:
per_pickup['Weighing Time (hours)'] = per_pickup['Total Quantity (lbs)'].apply(weigh_time)
display(per_pickup.head())

In [ ]:
weigh = per_pickup[['Agency', 'Total Quantity (lbs)', 'Weighing Time (hours)' ]]
display(weigh.head())


mean_weigh = per_pickup['Weighing Time (hours)'].mean()
median_weigh = per_pickup['Weighing Time (hours)'].median()
# Corrected the range calculation to use 'Cost Per Pound ($)' from per_pickup (costs) as it should be consistent
range_weigh = per_pickup['Weighing Time (hours)'].max() - per_pickup['Weighing Time (hours)'].min()

print(f"Mean Weighing Time (hours): {mean_weigh:.4f}")
print(f"Median Weighing Time (hours): {median_weigh:.4f}")
print(f"Range of Weighing Time (hours): {range_weigh:.4f}")

##**Calculating Labor Cost**

In [ ]:
#Labor Cost Function
##takes weighing time and driving time, returns labor cost assuming two workers
wage = 20.00 ##adjust to your local wage
def labor_cost(weighing_time, driving_time):
  labor_cost = ((weighing_time + (driving_time/60)) * wage) * 2
  return labor_cost

In [ ]:
per_pickup['Labor Cost ($)'] = per_pickup.apply(lambda row: labor_cost(row['Weighing Time (hours)'], row['Driving Duration (minutes)']), axis=1).round(2)
display(per_pickup.head())

##**Total Cost**




In [ ]:
per_pickup['Total Cost ($)'] = per_pickup['Labor Cost ($)'] + per_pickup['Cost Per Pickup ($)']
display(per_pickup.head())

In [ ]:
total = per_pickup[['Agency', 'Total Quantity (lbs)', 'Driving Distance (miles)', 'Weighing Time (hours)', 'Labor Cost ($)', 'Total Cost ($)']]
display(total.head())


mean_tot = per_pickup['Total Cost ($)'].mean()
median_tot = per_pickup['Total Cost ($)'].median()
# Corrected the range calculation to use 'Cost Per Pound ($)' from per_pickup (costs) as it should be consistent
range_tot = per_pickup['Total Cost ($)'].max() - per_pickup['Total Cost ($)'].min()

print(f"Mean of Total Cost ($): {mean_tot:.4f}")
print(f"Median of Total Cost ($): {median_tot:.4f}")
print(f"Range of Total Cost ($)): {range_tot:.4f}")

In [ ]:
agency_mean_total_cost = per_pickup.groupby('Agency')['Total Cost ($)'].mean().reset_index()
agency_mean_total_cost.rename(columns={'Total Cost ($)': 'Mean Total Cost ($)'}, inplace=True)
display(agency_mean_total_cost.round(2))

##**Cost per Pound, Adjusted to Include Labor Costs**


In [ ]:
cost_per_pound_with_labor_df = per_pickup.copy()
cost_per_pound_with_labor_df.head()

In [ ]:
grouped_cost_per_pound_with_labor_df = cost_per_pound_with_labor_df.groupby('Agency').agg(
    {"Total Quantity (lbs)": "sum",
     "Total Cost ($)": "sum",
     "Weighing Time (hours)": "sum",
     "Driving Duration (minutes)": "sum",
     "Driving Distance (miles)": "sum"
    }
)
grouped_cost_per_pound_with_labor_df['New Cost Per Pound ($)'] = grouped_cost_per_pound_with_labor_df['Total Cost ($)'] / grouped_cost_per_pound_with_labor_df['Total Quantity (lbs)']
grouped_cost_per_pound_with_labor_df
grouped_cost_per_pound_with_labor_df.loc[[
    'CrossStreets Neighborhood Services: Creekside',
    'Mercy Brown Bag Program',
    'San Lorenzo Family Help Center',
    'UC Berkeley',
    # 'Viola Blythe Community Service Center'
    ]]
# ['Total Cost ($)']

In [ ]:
grouped_cost_per_pound_with_labor_df.iloc[:, [0, 2, 3, 4]].loc[['CrossStreets Neighborhood Services: Creekside',
    'Mercy Brown Bag Program',
    'San Lorenzo Family Help Center',
    'UC Berkeley',]]

In [ ]:
grouped_cost_per_pound_with_labor_df = grouped_cost_per_pound_with_labor_df.drop(
    "Total", axis = 0
)

In [ ]:
grouped_cost_per_pound_with_labor_df

In [ ]:
per_pickup_with_labor_in_per_pound = per_pickup.copy()
per_pickup_with_labor_in_per_pound.head()

In [ ]:
per_pickup_with_labor_in_per_pound.columns

In [ ]:
per_pickup_with_labor_in_per_pound = per_pickup_with_labor_in_per_pound.rename(
    columns = {"Cost Per Pound ($)": "Cost Per Pound Without Labor ($)"}
)
per_pickup_with_labor_in_per_pound.head()

In [ ]:
per_pickup_with_labor_in_per_pound['Cost Per Pound With Labor ($)'] = per_pickup_with_labor_in_per_pound.apply(lambda row: per_pound_cost(row['Total Cost ($)'], row['Total Quantity (lbs)']), axis=1).round(6)

In [ ]:
per_pickup_with_labor_in_per_pound.head()

#**Data Analysis**

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with multiple subplots
fig, axes = plt.subplots(1, 2, figsize=(24, 7))
fig.suptitle('Relationships Between Driving Metrics and Quantity', fontsize=24)

# --- Scatter Plot 1: Total Quantity vs. Driving Distance ---
axes[0].scatter(per_pickup_with_labor_in_per_pound['Total Quantity (lbs)'], per_pickup_with_labor_in_per_pound['Driving Distance (miles)'], alpha=0.7, color='green')
axes[0].set_xlabel('Total Quantity (lbs)')
axes[0].set_ylabel('Driving Distance (miles)')
axes[0].set_title('Total Quantity vs. Driving Distance')
axes[0].grid(True, linestyle='--', alpha=0.6)

# --- Scatter Plot 2: Total Quantity vs. Driving Duration ---
axes[1].scatter(per_pickup_with_labor_in_per_pound['Total Quantity (lbs)'], per_pickup_with_labor_in_per_pound['Driving Duration (minutes)'], alpha=0.7, color='purple')
axes[1].set_xlabel('Total Quantity (lbs)')
axes[1].set_ylabel('Driving Duration (minutes)')
axes[1].set_title('Total Quantity vs. Driving Duration')
axes[1].grid(True, linestyle='--', alpha=0.6)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

#**Export Results!**

In [ ]:
per_pickup_with_labor_in_per_pound.to_csv('Cost-per-Pickup-and-Pound.csv', index=False)

In [ ]:
grouped_cost_per_pound_with_labor_df.to_csv('Cost-per-Pickup-and-Pound-per-agency.csv', index=False)